In [ ]:
import os
import random
import time
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import tyro
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
import wandb
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import gymnasium as gym
from environment import SimulationSettings, create_env
from rl_trainer import Agent, Args
from typing import List


In [30]:
import h5py

file_path = 'run/rl_train/env_benchmark.h5'

data = h5py.File(file_path, 'r')
data = data['data']
temperatures = data['temperatures'][:]
phis = data['phis'][:]
cpu_times = data['cpu_times'][:]



In [ ]:
plt.plot(temperatures[3000])

plt.show()

In [ ]:
temperatures


In [2]:
def set_integrator_heuristic(solver) -> List[str]:
    """Set integrator type based on temperature and equivalence ratio"""
    
    # Start with temperature-based decision
    integ = np.where(solver.T <= 600.0, 'boostRK', 'cvode')
    
    try:
        # Get equivalence ratio
        phi = solver.phi
        
        # Use boostRK for extreme conditions
        integ = np.where(phi == -1, 'boostRK', integ)  # invalid phi
        integ = np.where(phi <= 1e-8, 'boostRK', integ)  # oxidizer-dominated
        integ = np.where(phi >= 1e4, 'boostRK', integ)   # fuel-dominated
        
        # Create boolean mask for CVODE points
        cvode_mask = (integ == 'cvode')
        
        # Include neighboring points
        cvode_mask_left = np.roll(cvode_mask, 1)
        cvode_mask_right = np.roll(cvode_mask, -1)
        cvode_mask_left[0] = False
        cvode_mask_right[-1] = False
        
        use_cvode = cvode_mask | cvode_mask_left | cvode_mask_right
        integ = np.where(use_cvode, 'cvode', 'boostRK')
        
    except Exception as e:
        print(f"Warning: Could not calculate phi for integrator selection: {e}")
    
    return integ.tolist()


def integ_to_action(integ):
    integ = np.array(integ)
    action = np.zeros(len(integ))
    action[integ == 'boostRK'] = 1
    # convert to int
    action = action.astype(int)
    return action.tolist()


In [3]:
features_config = {
            'local_features': True,
            'neighbor_features': False,
            'gradient_features': False,
            'temporal_features': False,
            'window_size': 1
        }
    

reward_config = {
    'weights': {
        'accuracy': 0.4,
        'efficiency': 0.3,
        'stability': 0.3
            },
            'thresholds': {
                'time': 0.01,
                'error': 1e-3
            },
            'scaling': {
                'time': 0.1,
                'error': 1.0
            }
        }

In [4]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Environment setup
sim_settings = SimulationSettings(
    output_dir='run/diffusion_benchmark',
    t_end=0.06,
    n_points=100,
    global_timestep=1e-5
)

env = create_env(
    sim_settings=sim_settings,
    benchmark_file=f"{sim_settings.output_dir}/env_benchmark.h5",
    species_to_track=['CH4', 'O2', 'CO2', 'H2O'],
    features_config=features_config,
    reward_config=reward_config
)

# Agent and optimizer
agent = Agent(env).to(device)
optimizer = optim.Adam(agent.parameters(), lr=0.001, eps=1e-5)


In [ ]:
trained_model_path = "models/combustion_ppo_1d__1__1736867708/checkpoint_90.pt"
weights = torch.load(trained_model_path)
agent.load_state_dict(weights['model_state_dict'])


In [ ]:
obs, info = env.reset()
obs = torch.tensor(obs)

action, log_prob, entropy, value = agent.get_action_and_value(obs, deterministic=True)
print(action)

In [ ]:
plt.plot(action)
plt.plot(obs[:, 0])
plt.show()

In [ ]:
env.step(action)

In [ ]:
def run_full_simulation(env):
    obs, info = env.reset()
    cpu_times = []
    rewards = []
    done = False
    truncated = False
    while not done and not truncated:
        # action = set_integrator_heuristic(env.solver)
        # action = integ_to_action(action)
        action = [1] * len(env.solver.T)
        obs, reward, done, truncated, info = env.step(action)
        cpu_times.append(info['cpu_time'])
        rewards.append(reward)
    return cpu_times, rewards

cpu_times, rewards = run_full_simulation(env)
plt.plot(cpu_times)
plt.plot(rewards)
plt.show()

In [33]:
boostRK_rewards = np.array(rewards)
boostRK_cpu_times = np.array(cpu_times)

# save to file
np.savez(f"{sim_settings.output_dir}/boostRK_rewards.npz", rewards=boostRK_rewards, cpu_times=boostRK_cpu_times)


In [16]:
cvode_rewards = np.array(rewards)
cvode_cpu_times = np.array(cpu_times)

# save to file
np.savez(f"{sim_settings.output_dir}/cvode_rewards.npz", rewards=cvode_rewards, cpu_times=cvode_cpu_times)


In [14]:
heuristics_rewards = np.array(rewards)
heuristics_cpu_times = np.array(cpu_times)

# save to file
np.savez(f"{sim_settings.output_dir}/heuristics_rewards.npz", rewards=heuristics_rewards, cpu_times=heuristics_cpu_times)


In [ ]:
points_to_compare = 0, 500, 1000, 1500, 2000, 3000

fig, ax = plt.subplots(len(points_to_compare)//2, 2, figsize=(10, 10))

for i, point in enumerate(points_to_compare):
    ax[i//2, i%2].plot(boostRK_rewards[point], label='boostRK')
    ax[i//2, i%2].plot(cvode_rewards[point], label='cvode')
    ax[i//2, i%2].plot(heuristics_rewards[point], label='heuristics')
    ax[i//2, i%2].set_title(f'Rewards at point {point}')
    ax[i//2, i%2].legend()
    
plt.tight_layout()
plt.show()


fig = plt.figure(figsize=(10, 10), dpi=300)
plt.plot(cvode_cpu_times, label='cvode')
plt.plot(heuristics_cpu_times, label='heuristics')
plt.plot(boostRK_cpu_times, label='boostRK')
plt.legend()
plt.show()

In [ ]:
cummulative_cvode_cpu_times = np.cumsum(cvode_cpu_times)
cummulative_heuristics_cpu_times = np.cumsum(heuristics_cpu_times)
cummulative_boostRK_cpu_times = np.cumsum(boostRK_cpu_times)

print(f"cummulative cvode cpu time: {np.sum(cvode_cpu_times)}")
print(f"cummulative heuristics cpu time: {np.sum(heuristics_cpu_times)}")
print(f"cummulative boostRK cpu time: {np.sum(boostRK_cpu_times)}")


In [ ]:
cummulative_cvode_rewards = np.cumsum(cvode_rewards)
cummulative_heuristics_rewards = np.cumsum(heuristics_rewards)
cummulative_boostRK_rewards = np.cumsum(boostRK_rewards)

fig, ax = plt.subplots(2, 1, figsize=(10, 10))
ax[0].plot(cummulative_cvode_rewards, label='cvode')
ax[0].plot(cummulative_heuristics_rewards, label='heuristics')
ax[0].plot(cummulative_boostRK_rewards, label='boostRK')
ax[0].set_title('Cummulative Rewards')
ax[0].legend()

ax[1].plot(cummulative_cvode_cpu_times, label='cvode')
ax[1].plot(cummulative_heuristics_cpu_times, label='heuristics')
ax[1].plot(cummulative_boostRK_cpu_times, label='boostRK')
ax[1].set_title('Cummulative CPU Times')
ax[1].legend()
plt.tight_layout()
plt.show()


In [ ]:
total_cvode_cpu_time = np.sum(cvode_cpu_times)
total_heuristics_cpu_time = np.sum(heuristics_cpu_times)
total_boostRK_cpu_time = np.sum(boostRK_cpu_times)

total_cvode_reward = np.sum(cvode_rewards, axis=0)
total_heuristics_reward = np.sum(heuristics_rewards, axis=0)
total_boostRK_reward = np.sum(boostRK_rewards, axis=0)

fig = plt.figure(figsize=(10, 4), dpi=300)
plt.plot(total_cvode_reward, label='cvode')
plt.plot(total_heuristics_reward, label='heuristics')
plt.plot(total_boostRK_reward, label='boostRK')
plt.legend()
plt.show()


In [ ]:
total_cvode_reward


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
ax[0].plot(cvode_rewards[-4000], label='cvode')
ax[0].plot(heuristics_rewards[-4000], label='heuristics')
ax[1].plot(cvode_cpu_times, label='cvode')
ax[1].plot(heuristics_cpu_times, label='heuristics')

ax[0].set_title('Rewards')
ax[1].set_title('CPU Times')

ax[0].legend()
ax[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
import os
import random
import time
from dataclasses import dataclass
import torch
import numpy as np
import tyro
import wandb
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from environment import SimulationSettings, create_env
from ppo import PPO, RolloutBuffer

@dataclass
class Args:
    exp_name: str = "combustion_ppo_1d"
    seed: int = 1
    torch_deterministic: bool = True
    cuda: bool = False
    track: bool = False
    wandb_project_name: str = "combustion_control_1d"
    wandb_entity: str = None
    
    # Environment Parameters
    output_dir: str = 'run/rl_test'
    t_end: float = 0.06
    n_points: int = 100
    global_timestep: float = 1e-5
    
    # Algorithm specific arguments
    total_timesteps: int = 100000
    learning_rate: float = 2.5e-4
    num_envs: int = 1
    num_steps: int = 128
    gamma: float = 0.99
    gae_lambda: float = 0.95
    num_minibatches: int = 4
    update_epochs: int = 4
    eps_clip: float = 0.2
    entropy_coef: float = 0.01
    value_loss_coef: float = 0.5
    max_grad_norm: float = 0.5
    
    # Features configuration
    features_config: dict = None
    reward_config: dict = None

def make_env(args):
    """Create environment with specified settings"""
    sim_settings = SimulationSettings(
        output_dir=args.output_dir,
        t_end=args.t_end,
        n_points=args.n_points,
        global_timestep=args.global_timestep
    )
    
    env = create_env(
        sim_settings=sim_settings,
        benchmark_file=f"{args.output_dir}/env_benchmark.h5",
        species_to_track=['CH4', 'O2', 'CO2', 'H2O'],
        features_config=args.features_config,
        reward_config=args.reward_config
    )
    return env

def log_episode_info(writer, global_step, info):
    """Log episode information to tensorboard and wandb"""
    log_dict = {}
    
    if 'point_errors' in info:
        mean_error = np.mean(info['point_errors'])
        max_error = np.max(info['point_errors'])
        log_dict.update({
            "metrics/mean_error": mean_error,
            "metrics/max_error": max_error
        })
    
    if 'point_rewards' in info:
        mean_reward = np.mean(info['point_rewards'])
        min_reward = np.min(info['point_rewards'])
        log_dict.update({
            "metrics/mean_reward": mean_reward,
            "metrics/min_reward": min_reward
        })
    
    if 'cpu_time' in info:
        log_dict["metrics/step_time"] = info['cpu_time']
    
    if 'total_time' in info:
        log_dict["metrics/total_time"] = info['total_time']
    
    # Log to both tensorboard and wandb
    for key, value in log_dict.items():
        writer.add_scalar(key, value, global_step)
    if wandb.run is not None:
        wandb.log(log_dict, step=global_step)


def train(args):
    """Main training loop"""
    # Seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic
    
    # Setup run
    run_name = f"{args.exp_name}__{args.seed}__{int(time.time())}"
    if args.track:
        wandb.init(
            project=args.wandb_project_name,
            entity=args.wandb_entity,
            sync_tensorboard=True,
            config=vars(args),
            name=run_name,
            monitor_gym=True,
            save_code=True,
        )
    writer = SummaryWriter(f"runs/{run_name}")
    
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")
    
    # Environment setup
    env = make_env(args)
    
    # PPO agent setup
    state_dim = env.observation_space.shape[1]
    action_dim = len(env.integrator_options)
    
    ppo_agent = PPO(
        state_dim=state_dim,
        action_dim=action_dim,
        lr_actor=args.learning_rate,
        lr_critic=args.learning_rate,
        gamma=args.gamma,
        K_epochs=args.update_epochs,
        eps_clip=args.eps_clip,
        has_continuous_action_space=False
    )
    
    # Initialize environment
    obs, _ = env.reset(seed=args.seed)
    
    # Training loop
    start_time = time.time()
    global_step = 0
    
    # Calculate number of updates
    total_timesteps = args.total_timesteps
    num_updates = total_timesteps // args.num_steps
    
    for update in range(1, num_updates + 1):
        # Collect trajectory
        for step in range(0, args.num_steps):
            global_step += 1
            
            # Get actions for all points
            actions = []
            for point_obs in obs:
                action = ppo_agent.select_action(point_obs)
                actions.append(action)
            actions = np.array(actions)
            
            # Execute in environment
            next_obs, rewards, terminated, truncated, info = env.step(actions)
            
            # Store step data
            for point_idx in range(args.n_points):
                # Convert to done flag
                done = terminated or truncated
                
                # Store transition
                ppo_agent.buffer.rewards.append(rewards[point_idx])
                ppo_agent.buffer.is_terminals.append(done)
            
            # Log episode info
            if info:
                log_episode_info(writer, global_step, info)
            
            # Check if episode is done
            if terminated or truncated:
                print(f"Episode Done {update} - resetting environment")
                next_obs, _ = env.reset()
            
            obs = next_obs
        
        # Update PPO agent
        ppo_agent.update()
        
        # Log training metrics
        if wandb.run is not None:
            wandb.log({
                "charts/learning_rate": args.learning_rate,
                "train/mean_reward": np.mean(ppo_agent.buffer.rewards),
                "train/episode_length": len(ppo_agent.buffer.rewards)
            }, step=global_step)
        
        # Save model periodically
        if update % 10 == 0:
            save_path = f"models/{run_name}/checkpoint_{update}.pt"
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            ppo_agent.save(save_path)
            if args.track:
                wandb.save(save_path)
    
    # Save final model
    save_path = f"models/{run_name}/model_final.pt"
    ppo_agent.save(save_path)
    if args.track:
        wandb.save(save_path)
    
    # env.close()
    
    writer.close()
    return ppo_agent, env

def evaluate_policy(env, ppo_agent, num_episodes=4):
    """Evaluate the trained policy"""
    os.makedirs('evaluation', exist_ok=True)
    
    episode_rewards = np.zeros((num_episodes, env.sim_settings.n_points))
    episode_lengths = np.zeros(num_episodes)
    episode_errors = np.zeros((num_episodes, env.sim_settings.n_points))
    episode_cpu_times = np.zeros(num_episodes)
    
    for episode in range(num_episodes):
        obs, _ = env.reset()
        done = False
        episode_length = 0
        
        while not done:
            # Get actions for all points
            actions = []
            for point_obs in obs:
                action = ppo_agent.select_action(point_obs)
                actions.append(action)
            actions = np.array(actions)
            
            obs, rewards, terminated, truncated, info = env.step(actions)
            done = terminated or truncated
            
            episode_rewards[episode] += rewards
            episode_length += 1
            
            if done:
                print(f"Episode Done {episode}")
                episode_lengths[episode] = episode_length
                episode_errors[episode] = info['point_errors']
                episode_cpu_times[episode] = info['cpu_time']
                
                print(f"Cummulative Episode Reward: {np.sum(episode_rewards[episode])}")
                print(f"Cummulative Episode Error: {np.sum(episode_errors[episode])}")
                print(f"Cummulative Episode CPU Time: {np.sum(episode_cpu_times[episode])}")
    
    # Compute and return statistics
    results = {
        'mean_reward_per_point': np.mean(episode_rewards, axis=0),
        'std_reward_per_point': np.std(episode_rewards, axis=0),
        'mean_error_per_point': np.mean(episode_errors, axis=0),
        'mean_episode_length': np.mean(episode_lengths),
        'mean_cpu_time': np.mean(episode_cpu_times),
        'total_rewards': np.sum(episode_rewards),
        'max_error': np.max(episode_errors)
    }
    
    # Plot results
    plot_evaluation_results(results)
    
    return results

def plot_evaluation_results(results):
    """Plot evaluation results"""
    plt.figure(figsize=(10, 8))
    
    # Reward distribution
    plt.subplot(2, 1, 1)
    plt.plot(results['mean_reward_per_point'], label='Mean Reward')
    plt.fill_between(
        range(len(results['mean_reward_per_point'])),
        results['mean_reward_per_point'] - results['std_reward_per_point'],
        results['mean_reward_per_point'] + results['std_reward_per_point'],
        alpha=0.3
    )
    plt.title('Reward Distribution Across Grid Points')
    plt.xlabel('Grid Point')
    plt.ylabel('Reward')
    plt.legend()
    
    # Error distribution
    plt.subplot(2, 1, 2)
    plt.plot(results['mean_error_per_point'], 'r-', label='Mean Error')
    plt.yscale('log')
    plt.title('Error Distribution Across Grid Points')
    plt.xlabel('Grid Point')
    plt.ylabel('Error')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('evaluation/point_distributions.png')
    plt.close()



In [2]:
args = Args(cuda=False)

# Set default configurations if not provided
if args.features_config is None:
    args.features_config = {
        'local_features': True,
        'neighbor_features': False,
        'gradient_features': False,
        'temporal_features': False,
        'window_size': 1
    }

if args.reward_config is None:
    args.reward_config = {
        'weights': {
            'accuracy': 0.4,
            'efficiency': 0.3,
            'stability': 0.3
        },
        'thresholds': {
            'time': 0.01,
            'error': 1e-3
        },
        'scaling': {
            'time': 0.1,
            'error': 1.0
        }
    }


In [3]:
# Environment setup
env = make_env(args)

# PPO agent setup
state_dim = env.observation_space.shape[1]
action_dim = len(env.integrator_options)

ppo_agent = PPO(
    state_dim=state_dim,
    action_dim=action_dim,
    lr_actor=args.learning_rate,
    lr_critic=args.learning_rate,
    gamma=args.gamma,
    K_epochs=args.update_epochs,
    eps_clip=args.eps_clip,
    has_continuous_action_space=False
)



In [ ]:
trained_model_path = "models/combustion_ppo_1d__1__1736895993/checkpoint_140.pt"
ppo_agent.load(trained_model_path)

In [ ]:
# Initialize environment
obs, _ = env.reset(seed=args.seed)

# Training loop
start_time = time.time()
global_step = 0

# Calculate number of updates
total_timesteps = args.total_timesteps
num_updates = total_timesteps // args.num_steps


In [ ]:

actions = []
for point_obs in obs:
    action = ppo_agent.select_action(point_obs, deterministic=False)
    actions.append(action)
actions = np.array(actions)
print(actions)
# next_obs, rewards, terminated, truncated, info = env.step(actions)

In [15]:
def evaluate_policy(env, ppo_agent, num_episodes=4):
    """Evaluate the trained policy"""
    os.makedirs('evaluation', exist_ok=True)
    
    episode_rewards = np.zeros((num_episodes, env.sim_settings.n_points))
    episode_lengths = np.zeros(num_episodes)
    episode_errors = np.zeros((num_episodes, env.sim_settings.n_points))
    episode_cpu_times = np.zeros(num_episodes)
    
    
    for episode in range(num_episodes):
        obs, _ = env.reset()
        done = False
        episode_length = 0
        episode_actions = []
        while not done:
            # Get actions for all points
            actions = []
            for point_obs in obs:
                action = ppo_agent.select_action(point_obs, deterministic=True)
                actions.append(action)
            actions = np.array(actions)
            episode_actions.append(actions)
            print(f"Number of 0 actions: {np.sum(actions == 0)} - Number of 1 actions: {np.sum(actions == 1)}")
            obs, rewards, terminated, truncated, info = env.step(actions)
            done = terminated or truncated
            
            episode_rewards[episode] += rewards
            episode_length += 1
            
            if done:
                print(f"Episode Done {episode}")
                episode_lengths[episode] = episode_length
                episode_errors[episode] = info['point_errors']
                episode_cpu_times[episode] = info['cpu_time']
                
                print(f"Cummulative Episode Reward: {np.sum(episode_rewards[episode])}")
                print(f"Cummulative Episode Error: {np.sum(episode_errors[episode])}")
                print(f"Cummulative Episode CPU Time: {np.sum(episode_cpu_times[episode])}")
    
    # Compute and return statistics
    results = {
        'mean_reward_per_point': np.mean(episode_rewards, axis=0),
        'std_reward_per_point': np.std(episode_rewards, axis=0),
        'mean_error_per_point': np.mean(episode_errors, axis=0),
        'mean_episode_length': np.mean(episode_lengths),
        'mean_cpu_time': np.mean(episode_cpu_times),
        'total_rewards': np.sum(episode_rewards),
        'max_error': np.max(episode_errors)
    }
    
    # Plot results
    plot_evaluation_results(results)
    
    return results, episode_actions

def plot_evaluation_results(results):
    """Plot evaluation results"""
    plt.figure(figsize=(10, 8))
    
    # Reward distribution
    plt.subplot(2, 1, 1)
    plt.plot(results['mean_reward_per_point'], label='Mean Reward')
    plt.fill_between(
        range(len(results['mean_reward_per_point'])),
        results['mean_reward_per_point'] - results['std_reward_per_point'],
        results['mean_reward_per_point'] + results['std_reward_per_point'],
        alpha=0.3
    )
    plt.title('Reward Distribution Across Grid Points')
    plt.xlabel('Grid Point')
    plt.ylabel('Reward')
    plt.legend()
    
    # Error distribution
    plt.subplot(2, 1, 2)
    plt.plot(results['mean_error_per_point'], 'r-', label='Mean Error')
    plt.yscale('log')
    plt.title('Error Distribution Across Grid Points')
    plt.xlabel('Grid Point')
    plt.ylabel('Error')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('evaluation/point_distributions.png')
    plt.close()

In [ ]:
results, action_list = evaluate_policy(env, ppo_agent, num_episodes=1)


In [ ]:
results

In [ ]:
action_list = results[1]


points_to_compare = 0, 500, 1000, 1500, 2000, 3000

fig, ax = plt.subplots(len(points_to_compare)//2, 2, figsize=(10, 10))

for i, point in enumerate(points_to_compare):
    ax[i//2, i%2].plot(action_list[point], label='boostRK')
    ax[i//2, i%2].set_title(f'Actions at point {point}')
    ax[i//2, i%2].legend()
    
plt.tight_layout()
plt.show()

In [ ]:
len(action_list)